In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

ticker = 'AAPL'
data = yf.download(ticker, start="2015-01-01", end="2020-01-01")


[*********************100%%**********************]  1 of 1 completed


In [2]:

def backtest_strategy(symbol_data, rsi_period, bollinger_ma, bollinger_n_std_dev):
    df = symbol_data.copy()

    # RSI
    df['RSI'] = ta.momentum.RSIIndicator(close=df['Close'], window=rsi_period).rsi()

    # Bollinger Bands
    mavg = df['Close'].rolling(window=bollinger_ma).mean()

    rolling_std = df['Close'].rolling(window=20).std()
    df['Bollinger High'] = mavg + rolling_std * bollinger_n_std_dev
    df['Bollinger Low'] = mavg - rolling_std * bollinger_n_std_dev

    # Buy/Sell signals
    df['Signal'] = 0  # Neutral
    df['Position'] = None

    # Assume no transaction costs; leverage = 1 (no leverage)
    holding = False

    for i in range(len(df)):
        # The signal is current close < bollinger low and rsi < 30.
        # If both are satisfied and I am not holding, I long the stock
        if not holding:
            if (df['Close'].iloc[i] < df['Bollinger Low'].iloc[i] and df['RSI'].iloc[i] < 30):
                df['Signal'].iloc[i] = 1  # Buy
                df['Position'] = 1
                holding = True

        # Exit signal
        elif holding:
            if (df['Close'].iloc[i] > df['Bollinger High'].iloc[i] and df['RSI'].iloc[i] > 70):
                df['Signal'].iloc[i] = -1  # Sell
                df['Position'] = 0
                holding = False

    # Log Returns, Cumulative Returns, etc.
    df['Log Return'] = np.log(df['Close']/df['Close'].shift(1))
    df['Strategy Log Return'] = df['Signal'] * df['Log Return']


    df['Cumulative Market Returns'] = np.exp(df['Log Return'].cumsum())
    df['Cumulative Strategy Returns'] = np.exp(df['Strategy Log Return'].cumsum())

     # Annualized Return
    annualized_return = np.exp(df['Strategy Log Return'].sum()) ** (252/len(df)) - 1

    # Annualized Standard Deviation of Log returns
    annualized_std = df['Strategy Log Return'].std() * np.sqrt(252)

    # Calculate Beta
    cov_matrix = np.cov(df['Strategy Log Return'][1:], df['Log Return'][1:])
    beta = cov_matrix[0, 1] / cov_matrix[1, 1]

    # Calculate Alpha (assuming risk-free rate = 0)
    market_return = df['Log Return'].mean() * 252
    strategy_return = df['Strategy Log Return'].mean() * 252
    alpha = strategy_return - (beta * market_return)

    # Sharpe Ratio (assuming risk-free rate = 0)
    sharpe_ratio = annualized_return / annualized_std

    # Sortino Ratio
    df['Negative Return'] = 0
    df.loc[df['Strategy Log Return'] < 0, 'Negative Return'] = df['Strategy Log Return']**2
    downside_risk = np.sqrt(df['Negative Return'].mean() * 252)
    sortino_ratio = annualized_return / downside_risk

    # Maximum Drawdown
    df['Running Max'] = df['Cumulative Strategy Returns'].cummax()
    df['Drawdown'] = df['Cumulative Strategy Returns'] / df['Running Max'] - 1
    max_drawdown = df['Drawdown'].min()

    metrics = {
        'Annualized Return': annualized_return,
        'Volatility': annualized_std,
        'Beta': beta,
        'Alpha': alpha,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Max Drawdown': max_drawdown,
        'Final Cumulative Return': df['Cumulative Strategy Returns'].iloc[-1]
    }

    return df, metrics
